In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import ScratchLSTM as lstm

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional


import importlib
import ScratchLSTM
import ScratchEmbedding
import ScratchRNN
import ScratchDense
import Model

importlib.reload(Model)
importlib.reload(ScratchEmbedding)
importlib.reload(ScratchDense)
importlib.reload(ScratchLSTM)
importlib.reload(ScratchRNN)

<module 'ScratchRNN' from 'f:\\Repository\\Semester 6\\cnn-rnn-lstm-scratch-ml-tubes-2\\lstm\\ScratchRNN.py'>

In [48]:
path = "../nusax/datasets/sentiment/indonesian/"

train_data = pd.read_csv(path+"train.csv")
test_data = pd.read_csv(path+"test.csv")

# Text extraction
X_train_text = train_data['text'].astype(str).values
y_train = train_data['label'].values

X_test_text = test_data['text'].astype(str).values
y_test = test_data['label'].values

# Encoding
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Configure tokenizer
vocab_size = 2796
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train_text)

# Tokenization
X_train_seq = tokenizer.texts_to_sequences(X_train_text)
X_test_seq = tokenizer.texts_to_sequences(X_test_text)

# Padding
max_len = 77
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post', truncating='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post', truncating='post')

In [49]:
model = load_model('CNN/model_type_1_CNN.keras')
# model = load_model('LSTM/model_layer_2.keras')

new_model = Model.convertModel(model)

# embedding_weights = model.layers[0].get_weights()[0]
# lstm_weights = model.layers[1].get_weights()
# dense_weights_1 = model.layers[5].get_weights()
# dense_weights_2 = model.layers[6].get_weights()
model.summary()

for i, layer in enumerate(model.layers):
    try:
        print(f"Layer {i}: {layer.name} '{layer.activation.__name__}'")
    except:
        None

0 Embedding layer found
1 RNN layer found
3 Dense layer found
DenseScratch initialized with weights shape: 128
4 Dense layer found
DenseScratch initialized with weights shape: 64


Model: "sequential_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_17 (Embedding)        │ (None, 77, 128)        │       357,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_35 (SimpleRNN)       │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_17 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_34 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_35 (Dense)                │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,197,707 (4.57 MB)

 Trainable params: 399,235 (1.52 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 798,472 (3.05 MB)

Layer 1: simple_rnn_35 'tanh'
Layer 3: dense_34 'relu'
Layer 4: dense_35 'softmax'


In [50]:
print(new_model.predict(X_train_pad[0]))
print(model.predict(X_train_pad[0].reshape(1, -1)))

0 (77,)
1 (77, 128)
2 (1, 128)
3 (1, 64)
[[0.28750477 0.54752773 0.16496751]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
[[0.2875048  0.54752773 0.16496749]]
